Instalação de dependências

In [148]:
# !pip install pandas numpy matplotlib scikit-learn xgboost 

Importação dos dados armazenados no banco de dados 
  - Inicialmente utilizando apenas ofertas da Apple
  - Cada captura corresponde a até 10 ofertas, sendo igualmente dividas entre compra e venda 
  - Cada oferta contém preço e volume, além de um tipo associado
  - Ask indica uma oferta de compra
  - Bid indica uma oferta de venda
  - As capturas também acompanham as melhores ofertas negociadas, mas esses dados não nos interessam por hora

In [149]:
import os
import pandas as pd
import numpy as np

input_path = os.path.join('df_final.csv')
df = pd.read_csv(input_path, parse_dates=["File_Date"])
df = df[df['Company'] == 'APPLE INC COM']
df = df[["File_Date", "Price_of_One_Share", "Transaction_type", "Number_of_Shares_Sold" ]]
df.head(10) #print

,File_Date,Price_of_One_Share,Transaction_type,Number_of_Shares_Sold
0,2024-07-10 12:21:57,230.90,asks,1.0
1,2024-07-10 12:21:57,230.94,asks,1.0
2,2024-07-10 12:21:57,230.92,bids,7.0
3,2024-07-10 12:21:57,230.92,bids,5.0
4,2024-07-10 12:21:57,230.92,bids,3.0
5,2024-07-10 12:21:57,230.90,asks,400.0
6,2024-07-10 12:21:57,230.91,bids,1.0
7,2024-07-10 12:21:57,230.91,asks,52.0
8,2024-07-10 12:21:57,230.91,asks,48.0
9,2024-07-10 12:21:57,230.92,bids,1.0


Renomeando as colunas
  - Os nomes simples facilitarão na hora de aplicar a janela deslizante

In [150]:
df = df.rename(columns={
    'File_Date': 'Fdate', 
    'Price_of_One_Share': 'P', 
    'Transaction_type': 'Type', 
    'Number_of_Shares_Sold': 'S'
})

df #print

,Fdate,P,Type,S
0,2024-07-10 12:21:57,230.90,asks,1.0
1,2024-07-10 12:21:57,230.94,asks,1.0
2,2024-07-10 12:21:57,230.92,bids,7.0
3,2024-07-10 12:21:57,230.92,bids,5.0
4,2024-07-10 12:21:57,230.92,bids,3.0
...,...,...,...,...
718585,2024-07-19 17:59:30,224.45,asks,40.0
718586,2024-07-19 17:59:30,224.48,asks,1.0
718587,2024-07-19 17:59:30,224.45,asks,60.0
718588,2024-07-19 17:59:30,224.48,asks,60.0


Eliminando capturas sem dados

In [151]:
df = df.dropna().reset_index(drop=True)
df #print

,Fdate,P,Type,S
0,2024-07-10 12:21:57,230.90,asks,1.0
1,2024-07-10 12:21:57,230.94,asks,1.0
2,2024-07-10 12:21:57,230.92,bids,7.0
3,2024-07-10 12:21:57,230.92,bids,5.0
4,2024-07-10 12:21:57,230.92,bids,3.0
...,...,...,...,...
37785,2024-07-19 17:59:30,224.45,asks,40.0
37786,2024-07-19 17:59:30,224.48,asks,1.0
37787,2024-07-19 17:59:30,224.45,asks,60.0
37788,2024-07-19 17:59:30,224.48,asks,60.0


Realizando ordenação por data de captura e tipo de oferta
  - Isso é essencial para agrupar posteriormente esses dados

In [152]:
df = df.sort_values(by=['Fdate', 'Type'], ascending=[True, True]).reset_index(drop=True)
df.head(11) #print

,Fdate,P,Type,S
0,2024-07-10 12:21:57,230.90,asks,1.0
1,2024-07-10 12:21:57,230.94,asks,1.0
2,2024-07-10 12:21:57,230.90,asks,400.0
3,2024-07-10 12:21:57,230.91,asks,52.0
4,2024-07-10 12:21:57,230.91,asks,48.0
5,2024-07-10 12:21:57,230.92,bids,7.0
6,2024-07-10 12:21:57,230.92,bids,5.0
7,2024-07-10 12:21:57,230.92,bids,3.0
8,2024-07-10 12:21:57,230.91,bids,1.0
9,2024-07-10 12:21:57,230.92,bids,1.0


Unindo o horário de capturas iniciadas e finalizadas em momentos diferentes 
  - Todas as ofertas armazenarão o horário da captura, mesmo que tenham sido armazenadas segundos depois
  - A duração considerada é de até 5 segundos
  - A partir da duração considerada, qualquer oferta será associada a uma nova captura

In [153]:
df2 = df.copy()
threshold = 5 #segundos

df2['Fdate'] = pd.to_datetime(df2['Fdate'])
# Cria-se uma coluna "group" que identifica ofertas capturadas dentro de um intervalo de 5 segundos
# cumsum() garante que cada grupo tenha um valor único
df2['group'] = (df2['Fdate'] - df2['Fdate'].shift().fillna(df2['Fdate'].min())).dt.total_seconds().gt(threshold).cumsum()
# Todas as ofertas dentro de um mesmo grupo recebem a data da primeira oferta
df2['Fdate'] = df2.groupby('group').transform('first')['Fdate']
df2 = df2.drop('group', axis=1)

df2 #print

,Fdate,P,Type,S
0,2024-07-10 12:21:57,230.90,asks,1.0
1,2024-07-10 12:21:57,230.94,asks,1.0
2,2024-07-10 12:21:57,230.90,asks,400.0
3,2024-07-10 12:21:57,230.91,asks,52.0
4,2024-07-10 12:21:57,230.91,asks,48.0
...,...,...,...,...
37785,2024-07-19 17:59:30,224.44,bids,1.0
37786,2024-07-19 17:59:30,224.44,bids,25.0
37787,2024-07-19 17:59:30,224.44,bids,75.0
37788,2024-07-19 17:59:30,224.44,bids,76.0


Reordenam-se as transações para facilitar o treinamento do modelo
  - As ofertas de compra são ordenadas de forma decrescente
  - As ofertas de venda são ordenadas de forma crescente

In [154]:
df2 = df2.set_index(['Fdate', 'Type'])
df2.loc[(slice(None), "asks"), :] = df2.loc[(slice(None), "asks"), :].sort_values(['Fdate', 'P'], ascending=[True, True])
df2.loc[(slice(None), "bids"), :] = df2.loc[(slice(None), "bids"), :].sort_values(['Fdate', 'P'], ascending=[True, False])

df2 = df2.reset_index()
df2.head(15) #print

,Fdate,Type,P,S
0,2024-07-10 12:21:57,asks,230.90,1.0
1,2024-07-10 12:21:57,asks,230.90,400.0
2,2024-07-10 12:21:57,asks,230.91,52.0
3,2024-07-10 12:21:57,asks,230.91,48.0
4,2024-07-10 12:21:57,asks,230.94,1.0
5,2024-07-10 12:21:57,bids,230.92,7.0
6,2024-07-10 12:21:57,bids,230.92,5.0
7,2024-07-10 12:21:57,bids,230.92,3.0
8,2024-07-10 12:21:57,bids,230.92,1.0
9,2024-07-10 12:21:57,bids,230.91,1.0


Para que as capturas sejam agrupadas por horário, precisamos ter diferentes níveis para cada tipo de oferta.
  - Haverão 5 níveis de ask e 5 níveis de bid por captura
  - Como cada nível terá um valor único na coluna "Type", poderemos usá-los como colunas
  - Teremos então preços e shares para cada nível, totalizando 20 colunas por captura
  - Os indivíduos do novo dataframe representarão cada captura

In [155]:
df2['N'] = df2.groupby(['Fdate', 'Type']).cumcount() + 1
df2['Type'] = df2['Type'].str[0] + '_' + df2['N'].astype('str')

df2 = df2.drop('N', axis=1)
print(df2.head(5)) #print
print(df2.tail(5)) #print

                Fdate Type       P      S
0 2024-07-10 12:21:57  a_1  230.90    1.0
1 2024-07-10 12:21:57  a_2  230.90  400.0
2 2024-07-10 12:21:57  a_3  230.91   52.0
3 2024-07-10 12:21:57  a_4  230.91   48.0
4 2024-07-10 12:21:57  a_5  230.94    1.0
                    Fdate Type       P      S
37785 2024-07-19 17:59:30  b_1  224.45  100.0
37786 2024-07-19 17:59:30  b_2  224.44    1.0
37787 2024-07-19 17:59:30  b_3  224.44   25.0
37788 2024-07-19 17:59:30  b_4  224.44   75.0
37789 2024-07-19 17:59:30  b_5  224.44   76.0


In [156]:
df2 = df2.pivot(columns='Type', index='Fdate', values=['P', 'S'])
df2 #print


P                                                    \
Type                    a_1      a_2     a_3      a_4     a_5     b_1     b_2   
Fdate                                                                           
2024-07-10 12:21:57  230.90  230.900  230.91  230.910  230.94  230.92  230.92   
2024-07-10 12:23:02  230.87  230.870  230.87  230.870  230.88  230.88  230.88   
2024-07-10 12:25:41  230.85  230.850  230.85  230.850  230.85  230.87  230.85   
2024-07-10 12:26:37  230.93  230.935  230.94  230.960  230.96  230.95  230.93   
2024-07-10 12:27:32  230.92  230.920  230.93  230.935  230.95  230.91  230.91   
...                     ...      ...     ...      ...     ...     ...     ...   
2024-07-19 17:55:46  224.44  224.450  224.45  224.480  224.49  224.47  224.45   
2024-07-19 17:56:42  224.45  224.450  224.48  224.480  224.49  224.45  224.44   
2024-07-19 17:57:38  224.45  224.450  224.48  224.480  224.49  224.45  224.44   
2024-07-19 17:58:33  224.45  224.450  224.48  224.480  224.49  224.45  224.44   
2024-07-19 17:59:30  224.45  224.450  224.48  224.480  224.49  224.45  224.44   

                                                 S                       \
Type                    b_3     b_4     b_5    a_1    a_2    a_3    a_4   
Fdate                                                                     
2024-07-10 12:21:57  230.92  230.92  230.91    1.0  400.0   52.0   48.0   
2024-07-10 12:23:02  230.88  230.88  230.88  500.0   10.0  690.0  610.0   
2024-07-10 12:25:41  230.85  230.85  230.85  100.0  100.0  100.0   73.0   
2024-07-10 12:26:37  230.92  230.91  230.91    1.0    3.0   10.0    2.0   
2024-07-10 12:27:32  230.91  230.91  230.91  100.0    5.0   10.0    2.0   
...                     ...     ...     ...    ...    ...    ...    ...   
2024-07-19 17:55:46  224.44  224.44  224.44   76.0   40.0   60.0   60.0   
2024-07-19 17:56:42  224.44  224.44  224.44   40.0   60.0    1.0   60.0   
2024-07-19 17:57:38  224.44  224.44  224.44   40.0   60.0    1.0   60.0   
2024-07-19 17:58:33  224.44  224.44  224.44   40.0   60.0   60.0    1.0   
2024-07-19 17:59:30  224.44  224.44  224.44   40.0   60.0    1.0   60.0   

                                                              
Type                   a_5    b_1    b_2    b_3   b_4    b_5  
Fdate                                                         
2024-07-10 12:21:57    1.0    7.0    5.0    3.0   1.0    1.0  
2024-07-10 12:23:02   52.0  100.0  100.0    3.0  25.0   23.0  
2024-07-10 12:25:41   27.0  100.0   59.0  100.0  34.0  100.0  
2024-07-10 12:26:37  100.0   32.0    4.0   44.0   6.0   21.0  
2024-07-10 12:27:32    1.0  100.0  100.0   92.0  50.0    1.0  
...                    ...    ...    ...    ...   ...    ...  
2024-07-19 17:55:46    1.0   22.0  100.0    1.0  25.0   75.0  
2024-07-19 17:56:42    1.0  100.0    1.0   25.0  75.0   76.0  
2024-07-19 17:57:38    1.0  100.0    1.0   25.0  75.0   76.0  
2024-07-19 17:58:33    1.0  100.0   25.0   75.0   1.0   76.0  
2024-07-19 17:59:30    1.0  100.0    1.0   25.0  75.0   76.0  

[3779 rows x 20 columns]

Verificando que não há informações faltantes.<br>
O fato de haverem exatamente 20 colunas por captura indica que não há capturas irregulares (com mais ofertas), o que demonstra a eficácia dos técnicas de tratamento empregadas.
Capturas com menos ofertas possuiriam valores faltantes (NaN), o que não ocorre.  

In [157]:
df2[df2.isna().any(axis=1)] #print

Empty DataFrame
Columns: [(P, a_1), (P, a_2), (P, a_3), (P, a_4), (P, a_5), (P, b_1), (P, b_2), (P, b_3), (P, b_4), (P, b_5), (S, a_1), (S, a_2), (S, a_3), (S, a_4), (S, a_5), (S, b_1), (S, b_2), (S, b_3), (S, b_4), (S, b_5)]
Index: []

Os multi-índices são planificados para facilitar a manipulação dos dados

In [158]:
df2.columns = [''.join(col) for col in df2.columns]
df2 #print

,Pa_1,Pa_2,Pa_3,Pa_4,Pa_5,Pb_1,Pb_2,Pb_3,Pb_4,Pb_5,Sa_1,Sa_2,Sa_3,Sa_4,Sa_5,Sb_1,Sb_2,Sb_3,Sb_4,Sb_5
Fdate,,,,,,,,,,,,,,,,,,,,
2024-07-10 12:21:57,230.90,230.900,230.91,230.910,230.94,230.92,230.92,230.92,230.92,230.91,1.0,400.0,52.0,48.0,1.0,7.0,5.0,3.0,1.0,1.0
2024-07-10 12:23:02,230.87,230.870,230.87,230.870,230.88,230.88,230.88,230.88,230.88,230.88,500.0,10.0,690.0,610.0,52.0,100.0,100.0,3.0,25.0,23.0
2024-07-10 12:25:41,230.85,230.850,230.85,230.850,230.85,230.87,230.85,230.85,230.85,230.85,100.0,100.0,100.0,73.0,27.0,100.0,59.0,100.0,34.0,100.0
2024-07-10 12:26:37,230.93,230.935,230.94,230.960,230.96,230.95,230.93,230.92,230.91,230.91,1.0,3.0,10.0,2.0,100.0,32.0,4.0,44.0,6.0,21.0
2024-07-10 12:27:32,230.92,230.920,230.93,230.935,230.95,230.91,230.91,230.91,230.91,230.91,100.0,5.0,10.0,2.0,1.0,100.0,100.0,92.0,50.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-19 17:55:46,224.44,224.450,224.45,224.480,224.49,224.47,224.45,224.44,224.44,224.44,76.0,40.0,60.0,60.0,1.0,22.0,100.0,1.0,25.0,75.0
2024-07-19 17:56:42,224.45,224.450,224.48,224.480,224.49,224.45,224.44,224.44,224.44,224.44,40.0,60.0,1.0,60.0,1.0,100.0,1.0,25.0,75.0,76.0
2024-07-19 17:57:38,224.45,224.450,224.48,224.480,224.49,224.45,224.44,224.44,224.44,224.44,40.0,60.0,1.0,60.0,1.0,100.0,1.0,25.0,75.0,76.0


Lógica descartada

In [159]:
# The prices starts with P, the shares starts with S, P_n and S_n indicates a pair of an observation
# Get the tuples of prices and shares based in a row
# Asks prices are Pa and bids prices are Pb
def get_tuples(row):
    prices = row.filter(like='P').to_numpy()
    shares = row.filter(like='S').to_numpy()
    
    n = len(prices)
    Pa = prices[:n//2]
    Pb = prices[n//2:]
    Sa = shares[:n//2]
    Sb = shares[n//2:]
    
    return list(zip(Pa, Sa, ['A']*len(Pa))) + list(zip(Pb, Sb, ['B']*len(Pb)))

# Evaluate average price based on the tuples, considering the shares and prices
def avg_price(tuples):
    # total = sum([p * s for p, s, _ in tuples])
    # do with numpy
    total = np.sum([p * s for p, s, _ in tuples])
    shares = np.sum([s for _, s, _ in tuples])
    
    return total / shares

def avg_price_of_row(row):
    return avg_price(get_tuples(row))

row = df2.iloc[0]
print(row) #print
print(get_tuples(row)) #print
print(avg_price_of_row(row)) #print

# 230.90263969171482

Pa_1    230.90
Pa_2    230.90
Pa_3    230.91
Pa_4    230.91
Pa_5    230.94
Pb_1    230.92
Pb_2    230.92
Pb_3    230.92
Pb_4    230.92
Pb_5    230.91
Sa_1      1.00
Sa_2    400.00
Sa_3     52.00
Sa_4     48.00
Sa_5      1.00
Sb_1      7.00
Sb_2      5.00
Sb_3      3.00
Sb_4      1.00
Sb_5      1.00
Name: 2024-07-10 12:21:57, dtype: float64
[(230.9, 1.0, 'A'), (230.9, 400.0, 'A'), (230.91, 52.0, 'A'), (230.91, 48.0, 'A'), (230.94, 1.0, 'A'), (230.92, 7.0, 'B'), (230.92, 5.0, 'B'), (230.92, 3.0, 'B'), (230.92, 1.0, 'B'), (230.91, 1.0, 'B')]
230.90263969171482


Finalmente, aplica-se a janela deslizante nas capturas de ofertas
  - Cada linha do dataframe resultante contém X observações de ofertas anteriores, a observação atual e a próxima observação
  - A janela será igual a X + 1 = 3, de forma que 1 é logicamente o tamanho mínimo da janela

In [160]:
def get_renamed_df(df, i):
    df.columns = [f'{col}-{i}' for col in df.columns]
    return df

window = 3
df_lag = get_renamed_df(df2.copy(), 0)

for i in range(1, window):
  temp_df = get_renamed_df(df2.copy().shift(i), i)
  df_lag = pd.concat([df_lag, temp_df], axis=1)

temp_df = df2.copy().shift(-1)
df_lag = pd.concat([df_lag, temp_df], axis=1)

df_lag = df_lag.dropna()
df_lag.columns #print

Index(['Pa_1-0', 'Pa_2-0', 'Pa_3-0', 'Pa_4-0', 'Pa_5-0', 'Pb_1-0', 'Pb_2-0',
       'Pb_3-0', 'Pb_4-0', 'Pb_5-0', 'Sa_1-0', 'Sa_2-0', 'Sa_3-0', 'Sa_4-0',
       'Sa_5-0', 'Sb_1-0', 'Sb_2-0', 'Sb_3-0', 'Sb_4-0', 'Sb_5-0', 'Pa_1-1',
       'Pa_2-1', 'Pa_3-1', 'Pa_4-1', 'Pa_5-1', 'Pb_1-1', 'Pb_2-1', 'Pb_3-1',
       'Pb_4-1', 'Pb_5-1', 'Sa_1-1', 'Sa_2-1', 'Sa_3-1', 'Sa_4-1', 'Sa_5-1',
       'Sb_1-1', 'Sb_2-1', 'Sb_3-1', 'Sb_4-1', 'Sb_5-1', 'Pa_1-2', 'Pa_2-2',
       'Pa_3-2', 'Pa_4-2', 'Pa_5-2', 'Pb_1-2', 'Pb_2-2', 'Pb_3-2', 'Pb_4-2',
       'Pb_5-2', 'Sa_1-2', 'Sa_2-2', 'Sa_3-2', 'Sa_4-2', 'Sa_5-2', 'Sb_1-2',
       'Sb_2-2', 'Sb_3-2', 'Sb_4-2', 'Sb_5-2', 'Pa_1', 'Pa_2', 'Pa_3', 'Pa_4',
       'Pa_5', 'Pb_1', 'Pb_2', 'Pb_3', 'Pb_4', 'Pb_5', 'Sa_1', 'Sa_2', 'Sa_3',
       'Sa_4', 'Sa_5', 'Sb_1', 'Sb_2', 'Sb_3', 'Sb_4', 'Sb_5'],
      dtype='object')

Treinamento do modelo
  - Utiliza-se XGBoost para prever os preços e volumes de cada nível de asks e bids
  - A previsão é do tipo multioutput, ou seja, teremos 20 saídas para cada observação
  - A proporção entre os conjuntos de treino e teste é de 4:1
  - As métricas de avaliação são o R2 (coeficiente de determinação) e o RMSE (raiz do erro quadrático médio)

In [169]:
import xgboost as xgb
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error
from sklearn.model_selection import train_test_split

# train_size = int(len(df_lag)*0.8)
# train = df_lag[:train_size]
# test = df_lag[train_size:]

target_variables = [col for col in df_lag.columns if len(col) == 4]
features = [col for col in df_lag.columns if col not in target_variables]

X_train, X_test, y_train, y_test = train_test_split(
    df_lag[features],
    df_lag[target_variables],
    test_size=0.2,
    random_state=42
)

# X_train = train[features]
# y_train = train[target_variables]
# X_test = test[features]
# y_test = test[target_variables]

model = xgb.XGBRegressor()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

r2 = r2_score(y_test, y_pred, multioutput='raw_values')
mape = mean_absolute_percentage_error(y_test, y_pred, multioutput='raw_values')
mae = mean_absolute_error(y_test, y_pred, multioutput='raw_values')
rmse = np.sqrt(mean_squared_error(y_test, y_pred, multioutput='raw_values'))


Impressão dos resultados

In [170]:
results = pd.DataFrame({
    'R2': r2,
    'RMSE': rmse,
    'MAPE': mape,
    'MAE': mae
}, index=y_test.columns)

results #print

,R2,RMSE,MAPE,MAE
Pa_1,0.993648,0.299000,0.000606,0.139444
Pa_2,0.993836,0.294421,0.000610,0.140286
Pa_3,0.993572,0.300613,0.000617,0.142098
Pa_4,0.993787,0.295488,0.000605,0.139174
Pa_5,0.993654,0.298584,0.000613,0.141196
Pb_1,0.993794,0.295475,0.000590,0.135753
Pb_2,0.993749,0.296704,0.000597,0.137465
Pb_3,0.993841,0.294566,0.000604,0.139085
Pb_4,0.993666,0.298760,0.000615,0.141490
Pb_5,0.993571,0.300972,0.000617,0.142052
